In [2]:
import torch
import torch.nn as nn
from torchviz import make_dot
from torch.autograd import Variable

ModuleNotFoundError: No module named 'torchviz'

In [ ]:
class DanQ(nn.Module):
    def __init__(self, ):
        super(DanQ, self).__init__()
        self.Conv1 = nn.Conv1d(in_channels=4, out_channels=320, kernel_size=26)
        self.Conv1.weight.data = torch.Tensor(np.load('conv1_weights.npy'))
        self.Conv1.bias.data = torch.Tensor(np.load('conv1_bias.npy'))
        self.Maxpool = nn.MaxPool1d(kernel_size=13, stride=13)
        self.Drop1 = nn.Dropout(p=0.2)
        self.BiLSTM = nn.LSTM(input_size=320, hidden_size=320, num_layers=2,
                                 batch_first=True,
                                 dropout=0.5,
                                 bidirectional=True)
        self.Linear1 = nn.Linear(75*640, 925)
        self.Linear1.weight.data = torch.Tensor(np.load('linear11_weights.npy'))
        self.Linear1.bias.data = torch.Tensor(np.load('linear1_bias.npy'))
        self.Linear2 = nn.Linear(925, 919)
        self.Linear2.weight.data = torch.Tensor(np.load('linear22_weights.npy'))
        self.Linear2.bias.data = torch.Tensor(np.load('linear2_bias.npy'))

    def forward(self, input):
        x = self.Conv1(input)
        x = F.relu(x)
        x = self.Maxpool(x)
        x = self.Drop1(x)
        x_x = torch.transpose(x, 1, 2)
        x, (h_n,h_c) = self.BiLSTM(x_x)
        #x, h_n = self.BiGRU(x_x)
        x = x.contiguous().view(-1, 75*640)
        x = self.Linear1(x)
        x = F.relu(x)
        x = self.Linear2(x)
        return x

In [ ]:
x = Variable(torch.randn(100,4,1000))

In [ ]:
danq = DanQ()

In [ ]:
vis_graph = make_dot(danq(x),params=dict(danq.named_parameters()))

In [ ]:
vis_graph.view()